$$ \min f(x) \qquad \mbox{suj. a} \qquad \ell \leq x \leq u. $$

In [1]:
using Plots
gr()

Plots.GRBackend()

In [2]:
using Interact

In [11]:
function bisseccao(f::Function, a::Real, b::Real; tol=1e-6)
    (fa, fb) = (f(a), f(b))
    if abs(fa) < tol
        return a, fa, 0
    elseif abs(fb) < tol
        return b, fb, 0
    elseif fa * fb > 0
        error("f(a) e f(b) devem ter sinais diferentes")
    end
    escala = max(fa, fb)
    x = (a+b)/2
    fx = f(x)
    iter = 0
    while abs(fx)/escala > tol
        if fx * fa < 0
            b = x
        else
            a = x
        end
    
        x = (a+b)/2
        fx = f(x)
        iter = iter + 1
    end
    
    return x, fx, iter
end

bisseccao (generic function with 1 method)

In [42]:
@manipulate for μ = linspace(1e-6, 10.0, 1000)
    h(x) = -2*x*(x-1)*(2-x) - μ*(2-x) + μ*(x-1)
    x = bisseccao(h, 1, 2)[1]
    plot(x->-x^2 - μ*log(x-1) - μ*log(2-x), 1.00001, 1.9999)
    plot!(x->-x^2, 0, 3, c=:red)
    scatter!([x], [-x^2], c=:blue)
    ylims!(-5, 9)
end

Interact.Slider{Float64}(Signal{Float64}(4.994995495495496, nactions=0),"μ",4.994995495495496,linspace(1.0e-6,10.0,1000),true)

In [8]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
x = linspace(-0.5, 1.5, 100)
y = linspace(-0.5, 1.5, 100)
l = [-0.2 0.1]
u = [1.1 0.9]
contour(x, y, (x,y)->f([x;y]))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#f57d15; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,321.6 40.4899,322.763 45.5468,323.911 47.334,324.31 50.6037,325.03 55.6605,326.127 60.7174,327.209 64.8443,328.08 65.7743,328.272 70.8311,329.306 
 75.888,330.325 80.9449,331.328 83.6086,331.849 86.0017,332.309 91.0586,333.266 96.1155,334.208 101.172,335.136 103.845,335.618 106.229,336.041 111.286,336.924 
 116.343,337.793 121.4,338.646 125.865,339.387 126.457,339.484 131.514,340.295 136.57,341.092 141.627,341.874 146.684,342.641 150.139,343.156 151.741,343.391 
 156.798,344.118 161.855,344.831 166.912,345.53 171.969,346.214 177.025,346.884 177.341,346.925 182.082,347.531 187.139,348.162 192.196,348.78 197.253,349.383 
 202.31,349.973 207.367,350.548 208.684,350.694 212.423,351.103 217.48,351.642 222.537,352.167 227.594,352.678 232.651,353.175 237.708,353.659 242.765,354.128 
 246.487,354.463 247.822,354.582 252.878,355.017 257.935,355.438 262.992,355.846 268.049,356.24 273.106,356.62 278.163,356.987 283.22,357.34 288.276,357.679 
 293.333,358.005 297.025,358.232 298.39,358.315 303.447,358.609 308.504,358.889 313.561,359.156 318.618,359.41 323.675,359.649 328.731,359.876 333.788,360.088 
 338.845,360.288 343.902,360.473 348.959,360.646 354.016,360.804 359.073,360.95 364.129,361.081 369.186,361.2 374.243,361.304 379.3,361.395 384.357,361.473 
 389.414,361.537 394.471,361.588 399.528,361.625 404.584,361.649 409.641,361.659 414.698,361.656 419.755,361.639 424.812,361.608 429.869,361.564 434.926,361.507 
 439.983,361.436 445.039,361.352 450.096,361.254 455.153,361.142 460.21,361.017 465.267,360.879 470.324,360.727 475.381,360.561 480.437,360.382 485.494,360.19 
 490.551,359.984 495.608,359.764 500.665,359.531 505.722,359.285 510.779,359.025 515.836,358.751 520.892,358.464 524.789,358.232 525.949,358.162 531.006,357.844 
 536.063,357.511 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e65e2d; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 536.063,343.714 531.006,344.061 525.949,344.395 520.892,344.713 515.836,345.018 510.779,345.308 505.722,345.584 500.665,345.846 495.608,346.093 490.551,346.326 
 485.494,346.544 480.437,346.748 475.735,346.925 475.381,346.938 470.324,347.111 465.267,347.27 460.21,347.415 455.153,347.545 450.096,347.662 445.039,347.764 
 439.983,347.852 434.926,347.926 429.869,347.986 424.812,348.032 419.755,348.064 414.698,348.082 409.641,348.085 404.584,348.075 399.528,348.05 394.471,348.011 
 389.414,347.958 384.357,347.891 379.3,347.81 374.243,347.715 369.186,347.605 364.129,347.482 359.073,347.344 354.016,347.192 348.959,347.026 346.112,346.925 
 343.902,346.845 338.845,346.648 333.788,346.437 328.731,346.211 323.675,345.971 318.618,345.717 313.561,345.448 308.504,345.165 303.447,344.868 298.39,344.556 
 293.333,344.23 288.276,343.889 283.22,343.535 278.163,343.166 278.034,343.156 273.106,342.777 268.049,342.373 262.992,341.954 257.935,341.521 252.878,341.074 
 247.822,340.612 242.765,340.135 237.708,339.644 235.135,339.387 232.651,339.135 227.594,338.606 222.537,338.063 217.48,337.505 212.423,336.932 207.367,336.345 
 202.31,335.742 201.289,335.618 197.253,335.117 192.196,334.475 187.139,333.818 182.082,333.147 177.025,332.46 172.625,331.849 171.969,331.756 166.912,331.027 
 161.855,330.283 156.798,329.524 151.741,328.75 147.448,328.08 146.684,327.958 141.627,327.14 136.57,326.306 131.514,325.457 126.457,324.592 124.837,324.31 
 121.4,323.702 116.343,322.791 111.286,321.865 106.229,320.923 104.217,320.541 101.172,319.955 96.1155,318.964 91.0586,317.958 86.0017,316.936 85.2068,316.772 
 80.9449,315.882 75.888,314.809 70.8311,313.72 67.5523,313.003 65.7743,312.608 60.7174,311.466 55.6605,310.308 51.0376,309.234 50.6037,309.131 45.

Vamos considerar o problema com barreira

$$ \phi(x; \mu) = f(x) - \mu \log(x - l) - \mu \log(u - x). $$

In [9]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
axis = [-0.5, 1.5, -0.5, 1.5]

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = linspace(l[1]+1e-4, u[1]-1e-4, 100)
y = linspace(l[2]+1e-4, u[2]-1e-4, 100)

ϕ(x, μ) = f(x) - μ*(sum(log(x-l)) + sum(log(u-x)))

μ = 1.0
contour(x, y, (x,y)->ϕ([x;y], μ))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)
xlims!(axis[1], axis[2])
ylims!(axis[3], axis[4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#d24742; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 185.071,265.117 182.855,265.104 179.568,265.085 176.282,265.064 172.995,265.043 169.709,265.02 166.422,264.996 163.136,264.971 159.849,264.945 156.563,264.916 
 153.277,264.886 149.99,264.854 146.704,264.819 143.417,264.781 140.131,264.74 136.844,264.694 133.558,264.643 130.271,264.584 126.985,264.516 123.698,264.434 
 120.412,264.33 117.126,264.187 113.839,263.946 113.173,263.61 112.682,262.102 112.38,260.595 112.158,259.088 111.978,257.581 111.826,256.073 111.694,254.566 
 111.576,253.059 111.468,251.552 111.37,250.044 111.278,248.537 111.193,247.03 111.112,245.523 111.036,244.015 110.964,242.508 110.895,241.001 110.83,239.493 
 110.767,237.986 110.707,236.479 110.648,234.972 110.593,233.464 110.553,232.348 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ba3655; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 110.553,194.139 110.555,194.276 110.579,195.783 110.604,197.29 110.63,198.797 110.658,200.305 110.686,201.812 110.714,203.319 110.744,204.826 110.775,206.334 
 110.807,207.841 110.84,209.348 110.874,210.855 110.909,212.363 110.944,213.87 110.981,215.377 111.02,216.885 111.059,218.392 111.099,219.899 111.141,221.406 
 111.184,222.914 111.228,224.421 111.274,225.928 111.321,227.435 111.37,228.943 111.42,230.45 111.472,231.957 111.526,233.464 111.582,234.972 111.64,236.479 
 111.701,237.986 111.764,239.493 111.829,241.001 111.898,242.508 111.97,244.015 112.046,245.523 112.126,247.03 112.212,248.537 112.303,250.044 112.402,251.552 
 112.509,253.059 112.628,254.566 112.76,256.073 112.912,257.581 113.091,259.088 113.314,260.595 113.616,262.102 113.839,262.788 115.692,263.61 117.126,263.715 
 120.412,263.858 123.698,263.962 126.985,264.044 130.271,264.112 133.558,264.171 136.844,264.222 140.131,264.268 143.417,264.309 146.704,264.347 149.99,264.382 
 153.277,264.414 156.563,264.444 159.849,264.473 163.136,264.499 166.422,264.524 169.709,264.548 172.995,264.571 176.282,264.592 179.568,264.613 182.855,264.632 
 186.141,264.651 189.428,264.669 192.714,264.686 196.001,264.702 199.287,264.718 202.573,264.733 205.86,264.747 209.146,264.761 212.433,264.775 215.719,264.787 
 219.006,264.799 222.292,264.811 225.579,264.822 228.865,264.833 232.152,264.843 235.438,264.853 238.724,264.863 242.011,264.871 245.297,264.88 248.584,264.888 
 251.87,264.896 255.157,264.903 258.443,264.91 261.73,264.916 265.016,264.922 268.303,264.928 271.589,264.933 274.876,264.938 278.162,264.943 281.448,264.947 
 284.735,264.951 288.021,264.954 291.308,264.957 294.594,264.959 297.881,264.961 301.167,264.963 304.454,264.964 307.74,264.965 311.027,264.966 314.313,264.966 
 317.599,264.965 320.886,264.964 324.172,264.963 327.459,264.961 330.745,264.958 334.032,264.955 337.318,264.952 340.605,264.948 343.891,264.943 347.178,264.938 
 350.464,264.932 353.751,264.926 357.037,264.918 360.323,264.91 363.61,264.901 366.896,264.892 370.183,264.881 373.469,264.869 376.756,264.856 380.042,264.842 
 383.329,264.827 386.615,264.81 389.902,264.792 393.188,264.771 396.474,264.748 399.761,264.723 403.047,264.695 406.334,264.664 409.62,264.628 412.907,264.586 
 416.193,264.538 419.48,264.479 422.766,264.407 426.053,264.313 429.339,264.179 432.626,263.948 433.3,263.61 433.794,262.102 434.097,260.595 434.322,259.088 
 434.502,257.581 434.655,256.073 434.788,254.566 434.907,253.059 435.015,251.552 435.114,250.044 435.206,248.537 435.292,247.03 435.373,245.523 435.45,244.015 
 435.522,242.508 435.591,241.001 435.657,239.493 435.72,237.986 435.781,236.479 435.84,234.972 435.896,233.464 435.912,233.017 
 "/>
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#9e2a62; stroke-width:0.8; stroke-opa

In [10]:
using Interact

f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
axis = [-0.5, 1.5, -0.5, 1.5]

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = linspace(l[1]+1e-4, u[1]-1e-4, 100)
y = linspace(l[2]+1e-4, u[2]-1e-4, 100)

ϕ(x, μ) = f(x) - μ*(sum(log(x-l)) + sum(log(u-x)))

@manipulate for μ = linspace(1.0, 10.0, 1000)
    contour(x, y, (x,y)->ϕ([x;y], log(μ)), levels=30)
    plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)
    xlims!(axis[1], axis[2])
    ylims!(axis[3], axis[4])
end

Interact.Slider{Float64}(Signal{Float64}(5.495495495495495, nactions=0),"μ",5.495495495495495,linspace(1.0,10.0,1000),true)

As condições KKT desse problema são
$$ \nabla f(x) - \mu (X-L)^{-1}e + \mu (U-X)^{-1}e = 0, $$
onde $X$, $L$ e $U$ são matrizes diagonais com $x$, $\ell$ e $u$ na diagonal,
e $e$ é um vetor de uns. Note que, implicitamente $\ell < x < u$.

Agora, vamos definir vetores $z_L = \mu (X-L)^{-1}e$ e $z_U = \mu (U-X)^{-1}e$.
Desse modo, temos $(X-L)z_L = \mu e$ e $(U-X)z_U = \mu e$.
Ou ainda, $(X-L)Z_L e = \mu e$ e $(U-X)Z_U e = \mu e$.

Substituindo isso na condição KKT, temos
\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = \mu e, \\
(U-X)Z_U e & = \mu e, \\
\end{align*}
De antes, temos $\ell < x < u$, e agora temos $z_L, z_U > 0$.
Essas condições, quando $\mu \to 0$, são as condições KKT do sistema original.

\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = 0, \\
(U-X)Z_U e & = 0, \\
\end{align*}

Dessa maneira uma estratégia de resolução é usar um parâmetro $\sigma \in [0,1]$ para decidir o que seguir, usando as equações
\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = \sigma\mu e, \\
(U-X)Z_U e & = \sigma\mu e, \\
\end{align*}

Novamente, vamos tentar aplicar Newton nesse sistema. Além disso, temos a preocupação dos sinais,
que serão tratados usando uma condição impedindo-os de atravessar a parede.

\begin{align*}
\left[
\begin{array}{ccc}
H(x) & -I & I \\
Z_L & X-L & 0 \\
-Z_U & 0 & U-X
\end{array}
\right]
\left[
\begin{array}{c}
d_x \\ d_L \\ d_U
\end{array}
\right]
=
-\left[
\begin{array}{c}
\nabla f(x) - z_L + z_U \\
(X-L)z_L - \sigma\mu e \\
(U-X)z_U - \sigma\mu e
\end{array}
\right]
\equiv
-\left[
\begin{array}{c}
r_x \\ r_L \\ r_U
\end{array}
\right]
\end{align*}

In [15]:
using ForwardDiff

f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
∇f(x) = ForwardDiff.gradient(f, x)
H(x) = ForwardDiff.hessian(f, x)

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
zL = 1.2ones(2)
zU = 1.3ones(2)
μ = 10.0

M = [H(x)  -eye(2)  eye(2); [diagm(zL); -diagm(zU)]  diagm([x-l; u-x])]

6x6 Array{Float64,2}:
  2.0   0.0  -1.0  -0.0  1.0  0.0
  0.0   8.0  -0.0  -1.0  0.0  1.0
  1.2   0.0   0.2   0.0  0.0  0.0
  0.0   1.2   0.0   0.4  0.0  0.0
 -1.3  -0.0   0.0   0.0  1.1  0.0
 -0.0  -1.3   0.0   0.0  0.0  0.4

In [16]:
rx = ∇f(x) - zL + zU
rL = (x-l).*zL - μ
rU = (u-x).*zU - μ
d = -M\[rx; rL; rU]

6-element Array{Float64,1}:
  4.67327 
  0.280702
 20.7604  
 22.9579  
 13.3139  
 24.6123  

In [17]:
dx, dL, dU = d[1:2], d[3:4], d[5:6]

([4.673267326732671,0.280701754385964],[20.760396039603954,22.9578947368421],[13.313861386138612,24.612280701754386])

In [18]:
l .< x + dx .< u

2-element BitArray{1}:
 false
  true

Quanto até a barreira? O maior $\alpha > 0$ tal que
$$ \ell \leq x + \alpha d \leq u. $$
Mas na prática, não podemos ir pra barreira, então colocar um valor pequeno para dentro
da barreira, (i.e., não pode ser 0.0, mas pode ser 1e-4).
Por fim, pra deixar mais interessante, o valor tem que ser relativo à quanto falta para chegar lá,
uma vez que um passo "cego" poderia resultar num $\alpha$ negativo ou zero.
Isso resulta na condição abaixo, chamada **fração-para-a-fronteira**:

$$ \ell + (1-\tau)(x-l) \leq x + \alpha d \leq u - (1-\tau)(u-x), $$

ou seja,

$$ -\tau(x-l) \leq \alpha d \leq \tau (u-x), $$

para algum $\tau \in (0,1)$. Quanto mais perto de $1$, maior o passo possível.

Daí,
Se $d_i > 0$, então $\alpha \leq \tau \dfrac{u_i - x_i}{d_i}.$
Se $d_i < 0$, então $\alpha \leq \tau \dfrac{x_i - \ell_i}{d_i}.$
Por outro lado, gostaria da andar todo o passo de Newton, então $\alpha = 1$ é o ideal.

In [19]:
τ = 0.9
α = 1.0
for i = 1:2
    if dx[i] > 0
        α = min(α, τ*(u[i] - x[i])/d[i])
    elseif dx[i] < 0
        α = min(α, -τ*(x[i] - l[i])/d[i])
    end
    if dL[i] < 0
        α = min(α, -τ*zL[i]/dL[i])
    end
    if dU[i] < 0
        α = min(α, -τ*zU[i]/dU[i])
    end
end
α

0.21184322033898317

In [20]:
x + α*dx, zL + α*dL, zU + α*dU

([0.9900000000000001,0.5594647636039249],[5.597949152542374,6.063474353256023],[4.120451271186442,6.513944803746658])

In [55]:
using ForwardDiff

function ponto_interior_caixa(f, x, l, u; tol = 1e-5, maxiter = 1000, maxtime = 60, τ = 0.9)
    exit_flag = 0
    ∇f(x) = ForwardDiff.gradient(f, x)
    H(x) = ForwardDiff.hessian(f, x)
    
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    
    n = length(x)
    x = [l[i] < x[i] < u[i] ? x[i] : (l[i]+u[i])/2 for i = 1:n]
    fx = f(x)
    ∇fx = ∇f(x)
    Hx = H(x)
    zL = ones(n)
    zU = ones(n)
    
    μ = (dot(x-l,zL) + dot(u-x,zU))/2n    
    σ = 1.0
    
    rx = ∇fx - zL + zU
    rL = (x - l).*zL - μ*σ
    rU = (u - x).*zU - μ*σ
    
    
    while norm([rx;rL;rU]) > tol || μ > tol
        # Idealmente isso deveria ser esparso
        M = [Hx  -eye(n)  eye(n); [diagm(zL); -diagm(zU)]  diagm([x-l; u-x])]
        d = -M\[rx; rL; rU]
        dx, dL, dU = d[1:n], d[n+1:2n], d[2n+1:3n]
        if dot(dx, ∇fx) >= -1e-8
            dx = -rx
            dL = -rL
            dU = -rU
        end
        # Tamanho do passo

        α = 1.0
        for i = 1:n
            if dx[i] > 0
                α = min(α, τ*(u[i] - x[i])/dx[i])
            elseif dx[i] < 0
                α = min(α, -τ*(x[i] - l[i])/dx[i])
            end
            if dL[i] < 0
                α = min(α, -τ*zL[i]/dL[i])
            end
            if dU[i] < 0
                α = min(α, -τ*zU[i]/dU[i])
            end
        end
        
        x = x + α*dx
        zL = zL + α*dL
        zU = zU + α*dU
        μ = (dot(x-l, zL) + dot(u-x, zU))/2n
        σ *= 0.9
        
        fx = f(x)
        ∇fx = ∇f(x)
        Hx = H(x)
        
        rx = ∇fx - zL + zU
        rL = (x - l).*zL - μ*σ
        rU = (u - x).*zU - μ*σ
        
        iter = iter + 1
        if iter >= maxiter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= maxtime
            exit_flag = 2
            break
        end
    end
    return x, fx, rx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

ponto_interior_caixa (generic function with 1 method)

In [56]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
ponto_interior_caixa(f, x, l, u, maxiter=1000)

([0.9999924176556605,0.8999979338230314],0.04000165301614318,[5.288196096298048e-17,3.3306690738754696e-16],0,16,0.01214599609375)

In [57]:
xx = linspace(-0.5, 1.5, 100)
yy = linspace(-0.5, 1.5, 100)
l = [-0.2; 0.1]
u = [1.1; 0.9]
contour(xx, yy, (x,y)->f([x;y]))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

x = [0.0; 0.5]
scatter!([x[1]], [x[2]], c=:blue, leg=false)
xn = copy(x)
for i = 1:10
    xp = copy(xn)
    xn, _ = ponto_interior_caixa(f, x, l, u, maxiter=i)
    plot!([xp[1], xn[1]], [xp[2], xn[2]], c=:blue)
    scatter!([xn[1]], [xn[2]], c=:blue, ms=2)
end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#fb9c07; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 49.6018,331.686 54.3725,332.698 59.1431,333.697 63.1073,334.515 63.9137,334.679 68.6844,335.636 73.455,336.579 78.2256,337.509 81.1564,338.071 82.9963,338.419 
 87.7669,339.308 92.5375,340.184 97.3081,341.046 100.572,341.627 102.079,341.891 106.849,342.714 111.62,343.525 116.391,344.322 121.161,345.106 121.638,345.183 
 125.932,345.866 130.703,346.613 135.473,347.347 140.244,348.068 144.764,348.739 145.014,348.775 149.785,349.46 154.556,350.132 159.326,350.792 164.097,351.439 
 168.868,352.073 170.568,352.294 173.638,352.689 178.409,353.288 183.179,353.876 187.95,354.451 192.721,355.013 197.491,355.563 200.044,355.85 202.262,356.096 
 207.033,356.614 211.803,357.118 216.574,357.611 221.345,358.091 226.115,358.559 230.886,359.014 235.105,359.406 235.656,359.456 240.427,359.881 245.198,360.293 
 249.968,360.693 254.739,361.081 259.51,361.457 264.28,361.821 269.051,362.172 273.821,362.511 278.592,362.838 280.467,362.962 283.363,363.15 288.133,363.448 
 292.904,363.734 297.675,364.009 302.445,364.271 307.216,364.521 311.986,364.759 316.757,364.985 321.528,365.199 326.298,365.401 331.069,365.591 335.84,365.769 
 340.61,365.934 345.381,366.088 350.152,366.23 354.922,366.359 359.693,366.477 361.531,366.517 
 "/>
 
<polyline clip-path="url(#clip02)" style="stroke:#f57d15; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 521.894,350.326 517.124,350.635 512.353,350.932 507.582,351.215 502.812,351.486 498.041,351.744 493.27,351.99 488.5,352.222 486.936,352.294 483.729,352.44 
 478.959,352.644 474.188,352.836 469.417,353.015 464.647,353.181 459.876,353.335 455.105,353.477 450.335,353.605 445.564,353.722 440.794,353.825 436.023,353.916 
 431.252,353.995 426.482,354.061 421.711,354.114 416.94,354.155 412.17,354.183 407.399,354.199 402.628,354.202 397.858,354.193 393.087,354.171 388.317,354.136 
 383.546,354.089 378.775,354.029 374.005,353.957 369.234,353.872 364.463,353.775 359.693,353.665 354.922,353.542 350.152,353.407 345.381,353.26 340.61,353.1 
 335.84,352.927 331.069,352.742 326.298,352.544 321.528,352.333 320.696,352.294 316.757,352.108 311.986,351.869 307.216,351.617 302.445,351.352 297.675,351.075 
 292.904,350.785 288.133,350.483 283.363,350.167 278.592,349.839 273.821,349.498 269.051,349.144 264.28,348.778 263.784,348.739 259.51,348.394 254.739,347.996 
 249.968,347.586 245.198,347.162 240.427,346.726 235.656,346.277 230.886,345.814 226.115,345.339 224.586,345.183 221.345,344.846 216.574,344.338 211.803,343.816 
 207.033,343.281 202.262,342.734 197.491,342.173 192.956,341.627 192.721,341.598 187.95,341.002 183.179,340.393 178.409,339.77 173.638,339.135 168.868,338.485 
 165.886,338.071 164.097,337.819 159.326,337.133 154.556,336.433 149.785,335.72 145.014,334.993 141.935,334.515 140.244,334.249 135.473,333.484 130.703,332.705 
 125.932,331.912 121.161,331.105 120.315,330.96 116.391,330.274 111.62,329.427 106.849,328.566 102.079,327.691 100.537,327.404 97.3081,326.793 92.5375,325.875 
 87.7669,324.944 82.9963,323.998 82.2501,323.848 78.2256,323.025 73.455,322.036 68.6844,321.032 65.2221,320.292 63.9137,320.009 59.1431,318.959 54.3725,317.894 
 49.6018,316.815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e65e2d; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 49.6018,300.821 54.3725,301.997 56.4995,302.514 59.1431,303.145 63.9137,304.269 68.6844,305.377 71.7065,306.069 73.455,306.463 78.2256,307.521 82.9963,308.564 
 87.7669,309.592 87.9255,309.625 92.5375,310.587 97.3081,311.567 102.079,312.531 105.343,313.181 106.849,313.476 111.62,314.394 116.391,315.298 121.161,316.187 
 124.161,316.737 125.932,317.056 130.703,317.901 135.473,318.731 140.244,319.547 144.682,320.292 145.014,320.347 149.785,321.121 154.556,321.881 159.326,322.626 
 164.097,323.356 167

In [58]:
f(x) = -x[1]^2
l = [1.0]
u = [2.0]
x = [1.1]
ponto_interior_caixa(f, x, l, u, maxiter=1000)

([2.0],-4.0,[-2.1870792216727146],1,1000,0.03297591209411621)

In [59]:
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2
l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
ponto_interior_caixa(f, x, l, u, maxiter=1000)

([-0.05022336511562245,0.1],2.053157631880915,[-2.1690155093633154,13.098586886202044],1,1000,0.032160043716430664)

In [65]:
xx = linspace(-0.5, 1.5, 100)
yy = linspace(-0.5, 1.5, 100)
l = [-0.2; 0.1]
u = [1.1; 0.9]
contour(xx, yy, (x,y)->f([x;y]), levels=100)
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

x = [0.0; 0.5]
scatter!([x[1]], [x[2]], c=:blue, leg=false)
xn = copy(x)
for i = 1:20
    xp = copy(xn)
    xn, _ = ponto_interior_caixa(f, x, l, u, maxiter=i)
    plot!([xp[1], xn[1]], [xp[2], xn[2]], c=:blue)
    scatter!([xn[1]], [xn[2]], c=:blue, ms=2)
end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0a0622; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 49.6018,320.115 49.8378,320.292 54.3725,323.708 54.566,323.848 59.1431,327.154 59.5039,327.404 63.9137,330.452 64.679,330.96 68.6844,333.604 70.1231,334.515 
 73.455,336.611 75.8733,338.071 78.2256,339.478 81.9738,341.627 82.9963,342.207 87.7669,344.786 88.5405,345.183 92.5375,347.206 95.7126,348.739 97.3081,349.497 
 102.079,351.638 103.632,352.294 106.849,353.627 111.62,355.487 112.623,355.85 116.391,357.182 121.161,358.747 123.355,359.406 125.932,360.16 130.703,361.423 
 135.473,362.558 137.411,362.962 140.244,363.534 145.014,364.36 149.785,365.048 154.556,365.594 159.326,365.997 164.097,366.254 168.868,366.363 173.638,366.325 
 178.409,366.139 183.179,365.807 187.95,365.329 192.721,364.709 197.491,363.949 202.262,363.053 202.682,362.962 207.033,361.987 211.803,360.785 216.574,359.457 
 216.741,359.406 221.345,357.952 226.115,356.327 227.406,355.85 230.886,354.539 235.656,352.625 236.426,352.294 240.427,350.544 244.346,348.739 245.198,348.34 
 249.968,345.975 251.486,345.183 254.739,343.465 258.064,341.627 259.51,340.82 264.221,338.071 264.28,338.037 269.051,335.087 269.937,334.515 273.821,331.997 
 275.356,330.96 278.592,328.764 280.516,327.404 283.363,325.388 285.449,323.848 288.133,321.867 290.181,320.292 292.904,318.202 294.735,316.737 297.675,314.392 
 299.132,313.181 302.445,310.44 303.39,309.625 307.216,306.346 307.526,306.069 311.517,302.514 311.986,302.098 315.381,298.958 316.757,297.698 319.158,295.402 
 321.528,293.161 322.859,291.846 326.298,288.493 326.497,288.291 329.994,284.735 331.069,283.655 333.426,281.179 335.84,278.684 336.819,277.623 340.145,274.068 
 340.61,273.573 343.359,270.512 345.381,268.303 346.557,266.956 349.709,263.4 350.152,262.905 352.751,259.845 354.922,257.346 355.797,256.289 358.774,252.733 
 359.693,251.649 361.685,249.177 364.463,245.818 364.618,245.622 367.419,242.066 369.234,239.819 370.238,238.51 373.004,234.954 374.005,233.687 375.717,231.399 
 378.447,227.843 378.775,227.418 381.064,224.287 383.546,220.998 383.737,220.731 386.283,217.176 388.317,214.423 388.878,213.62 391.382,210.064 393.087,207.707 
 393.906,206.508 396.367,202.953 397.858,200.848 398.828,199.397 401.242,195.841 402.628,193.845 403.649,192.285 406.015,188.73 407.399,186.698 408.376,185.174 
 410.692,181.618 412.17,179.405 413.013,178.062 415.278,174.507 416.94,171.97 417.567,170.951 419.779,167.395 421.711,164.392 422.044,163.839 424.202,160.284 
 426.446,156.728 426.482,156.672 428.552,153.172 430.737,149.616 431.252,148.787 432.835,146.061 434.961,142.505 436.023,140.764 437.057,138.949 439.122,135.393 
 440.794,132.606 441.224,131.837 443.228,128.282 445.314,124.726 445.564,124.302 447.284,121.17 449.305,117.614 450.335,115.839 451.295,114.059 453.251,110.503 
 455.105,107.252 455.267,106.947 457.158,103.391 459.124,99.8357 459.876,98.4976 461.032,96.2799 462.929,92.7242 464.647,89.6176 464.877,89.1684 466.707,85.6127 
 468.61,82.0569 469.417,80.5744 470.462,78.5012 472.293,74.9454 474.188,71.4123 474.199,71.3897 475.961,67.8339 477.793,64.2782 478.959,62.0721 479.617,60.7224 
 481.375,57.1666 483.205,53.6109 483.729,52.6053 484.953,50.0551 486.706,46.4994 488.5,43.0043 488.528,42.9436 490.209,39.3879 491.955,35.8321 493.27,33.2323 
 493.715,32.2764 495.385,28.7206 497.122,25.1649 498.041,23.3242 498.827,21.6091 500.485,18.0534 502.21,14.4976 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#07051b; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 494.829,14.4976 493.27,17.8771 493.181,18.0534 491.374,21.6091 489.654,25.1649 488.5,27.6276 487.935,28.7206 486.126,32.2764 484.402,35.8321 483.729,37.2446 
 482.606,39.3879 480.798,42.9436 479.072,46.4994 478.959,46.7336 477.193,50.0551 475.388,53.6109 474.188,56.0489 473.584,57.1666 471.696,60.